In [8]:
import pandas as pd
import numpy as np
import random

# Set random seed for reproducibility
np.random.seed(42)

# Parameters for synthetic data
num_records = 10000  # Number of rows
start_date = "2020-01-01"
end_date = "2024-12-31"

# Generate random dates with seasonality
dates = pd.date_range(start=start_date, end=end_date, freq="D")
random_dates = np.random.choice(dates, num_records)
# Convert random_dates to pandas DatetimeIndex
random_dates = pd.to_datetime(random_dates)
# Calculate seasonality factor
seasonality_factor = (np.sin(2 * np.pi * (random_dates.dayofyear / 365)) + 1) / 2


# Generate synthetic data
data = {
    "Transaction_ID": [f"TXN{str(i).zfill(6)}" for i in range(1, num_records + 1)],
    "Date": random_dates,
    "Region": np.random.choice(["North", "South", "East", "West", "Central"], num_records),
    "Product": np.random.choice(
        ["Electronics", "Home Essentials", "Groceries", "Clothing", "Furniture"], num_records
    ),
    "Customer_Age": np.random.randint(18, 70, num_records),
    "Customer_Gender": np.random.choice(["Male", "Female", "Non-Binary"], num_records),
    "Customer_Segment": np.random.choice(["Retail", "Wholesale", "Enterprise"], num_records),
    "Sales_Channel": np.random.choice(["Online", "Offline", "Mixed"], num_records),
    "Units_Sold": np.random.poisson(5, num_records) + 1,  # Poisson distribution for count data
    "Unit_Price": np.random.uniform(20, 1000, num_records).round(2),  # Random unit price
}

# Add Revenue, Discounts, and Taxes
data["Discount"] = (np.random.uniform(0.05, 0.3, num_records) * data["Unit_Price"]).round(2)
data["Tax"] = (np.random.uniform(0.05, 0.2, num_records) * data["Unit_Price"]).round(2)
data["Revenue"] = (
    (data["Units_Sold"] * (data["Unit_Price"] - data["Discount"])) * (1 + seasonality_factor)
).round(2)
data["Expenses"] = (data["Revenue"] * np.random.uniform(0.5, 0.8, num_records)).round(2)
data["Profit"] = (data["Revenue"] - data["Expenses"] - data["Tax"]).round(2)

# Add anomalies for testing
# Convert data dictionary to DataFrame
df = pd.DataFrame(data)

# Add anomalies
anomaly_indices = np.random.choice(df.index, int(len(df) * 0.02), replace=False)  # 2% anomalies
df.loc[anomaly_indices, "Revenue"] *= np.random.uniform(1.5, 3, len(anomaly_indices))  # Inflate revenue anomalies
df.loc[anomaly_indices, "Profit"] *= np.random.uniform(-2, -1, len(anomaly_indices))  # Negative profit anomalies


# Add calculated fields
data["Profit_Margin"] = (data["Profit"] / data["Revenue"]).round(2)
# Ensure 'Date' is a pandas Series of datetime objects
df["Date"] = pd.to_datetime(df["Date"])

# Add calculated fields
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day_of_Week"] = df["Date"].dt.day_name()

# Create DataFrame
df = pd.DataFrame(data)

# Add a column for operational inefficiencies
df["Operational_Inefficiency"] = np.where(
    df["Expenses"] > df["Revenue"] * 0.7, "High", "Normal"
)

# Save to CSV (optional)
df.to_csv("globalmart_data.csv", index=False)

# Display the first few rows
print(df.head())


  Transaction_ID       Date   Region          Product  Customer_Age  \
0      TXN000001 2023-01-31    North        Groceries            30   
1      TXN000002 2023-12-30  Central  Home Essentials            21   
2      TXN000003 2022-05-10  Central        Furniture            40   
3      TXN000004 2023-07-18     East        Furniture            25   
4      TXN000005 2023-02-04  Central        Groceries            30   

  Customer_Gender Customer_Segment Sales_Channel  Units_Sold  Unit_Price  \
0          Female       Enterprise        Online           6      472.54   
1          Female           Retail         Mixed           6      622.40   
2            Male        Wholesale         Mixed           6      693.79   
3            Male           Retail        Online           4      109.37   
4            Male        Wholesale       Offline           3      712.07   

   Discount     Tax  Revenue  Expenses   Profit  Profit_Margin  \
0     38.76   64.35  4565.97   2405.68  2095.94   